# VisualOdometrySLAM

In [30]:
import random

import cv2
import numpy as np
import pandas as pd

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    cv2.setRNGSeed(seed)

In [31]:
SEED = 42
seed_all(SEED)

In [32]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_PATH_VIDEO = DATA_PATH /Path('visual_odometry/')
DATA_PATH_VIDEO.mkdir(parents=True, exist_ok=True)

DATA_PATH_OUTPUT = DATA_PATH / Path('output_data/')
DATA_PATH_OUTPUT.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = DATA_PATH / Path('models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

DATA_IMGS = Path('../imgs')
DATA_IMGS.mkdir(parents=True, exist_ok=True)

In [33]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_path)

# Основной код

In [34]:
from visual_slam.calibration import UniversalCalibration
from visual_slam.source import DatasetSource
from visual_slam.camera import PinholeCamera
from visual_slam.config import Config
from visual_slam.map.map import Map
from visual_slam.map.keyframe import KeyFrame
from visual_slam.map.map_point import MapPoint
from visual_slam.feature.tracker import FeatureTracker
from visual_slam.slam import SLAM
from visual_slam.viz.map_viz import MapVisualizer


VIDEO_PATH = DATA_PATH / 'KITTI_sequence_1/image_l'
CALIBRATION_PATH = DATA_PATH / 'KITTI_sequence_1/calib.txt'

calibration = UniversalCalibration().load_from(path=CALIBRATION_PATH)
video_source = DatasetSource(path=VIDEO_PATH)

In [35]:
# video_source.show(fps=2)

In [36]:
from visual_slam.config import Config, CameraConfig, FeatureConfig, TrackingConfig, MapConfig, LoopClosingConfig, OptimizationConfig, LocalMappingConfig

config = Config(
    camera=CameraConfig(
        width=1241,
        height=376,
        fx=718.856,
        fy=718.856,
        cx=607.1928,
        cy=185.2157,
    ),
    features=FeatureConfig(
        detector="fast_orb_anms",
        matcher="bf-hamming",
        detector_params={
            "nfeatures": 500,
            "use_anms": True,
            "anms_count": 2000,
            "anms_tolerance": 0.3,
        },
        matcher_params={
            "ratio_thresh": 0.75,
            "cross_check": False
        },
        filtered_params={
            "use_grid": False,
            "use_nms": False,            
            
            "use_ransac_fund_matrix": True,
            "use_orientation": True
        }
    ),
    tracking=TrackingConfig(
        min_inliers=10,
        min_parallax_deg=0.5,
        keyframe_interval=1,
        min_inlier_ratio=0.25,
        max_reprojection_error=5.0,
        use_ransac=True,
        
        max_translation_for_kf=2.0,
        max_rotation_for_kf=10.0,
        min_matches_for_kf=30,
    ),
    local_mapping=LocalMappingConfig(
        run_timeout=0.5,
        max_neighbors=5
    ),
    map=MapConfig(

    ),
    optimization=OptimizationConfig(
        lr=1e-4,
        n_iter=10,
        batch_size=64,
        huber_delta=5.0
    ),
    loop_closing=LoopClosingConfig(
        
    ),
    debug=True
)

In [37]:
config.features

FeatureConfig(detector='fast_orb_anms', matcher='bf-hamming', detector_params={'nfeatures': 500, 'use_anms': True, 'anms_count': 2000, 'anms_tolerance': 0.3}, matcher_params={'ratio_thresh': 0.75, 'cross_check': False}, filtered_params={'use_grid': False, 'use_nms': False, 'use_ransac_fund_matrix': True, 'use_orientation': True})

In [38]:
from time import sleep
from typing import Union

class Processing:
    
    def __init__(
        self, 
        video_path: Union[str, Path],
        calibration_file: Union[str, Path],                   
        config: Config,
    ):
        
        self.config = config
        
        self.video_source = DatasetSource(video_path)
        self.calibration = UniversalCalibration().load_from(calibration_file)
        K = self.calibration.mono.K
        h, w, _ = video_source.get_frame_shape()
        self.camera = PinholeCamera(
            width=w,
            height=h,
            fx=K[0, 0],
            fy=K[1, 1],
            cx=K[0, 2],
            cy=K[1, 2],
            dist_coeffs=self.calibration.mono.D,
        )
        
        self.slam = SLAM(
            camera=self.camera,
            config=self.config,
        )    
    
    def run(self):
        print("=== Запуск SLAM ===")

        total_frames = self.video_source.num_frames()
        print(f"Всего кадров: {total_frames}")

        count = 0
        max_cycles = 20
        while self.video_source.is_ok():
            img, timestamp = self.video_source.get_frame()
            if img is None:
                break

            result = self.slam.track([img], timestamp)
            count += 1
            print(f"#{count} Timestamp={timestamp:.2f} → Result={result}")

            if count >= max_cycles:
                print(f"Остановлено: достигнуто {max_cycles} циклов.")
                break
            sleep(1.0)
        self.slam.shutdown()
        print("=== SLAM завершён ===")
        
        
        
    

In [39]:
process = Processing(
    video_path=VIDEO_PATH,
    calibration_file=CALIBRATION_PATH,
    config=config
)
process.run()

=== Запуск SLAM ===
Всего кадров: 51
#1 Timestamp=0.00 → Result=<StateItem NOT_INITIALIZED (1): Система не инициализирована>
#2 Timestamp=1.00 → Result=<StateItem OK (3): Трекинг в норме>
#3 Timestamp=2.00 → Result=<StateItem OK (3): Трекинг в норме>
#4 Timestamp=3.00 → Result=<StateItem OK (3): Трекинг в норме>
#5 Timestamp=4.00 → Result=<StateItem OK (3): Трекинг в норме>
#6 Timestamp=5.00 → Result=<StateItem OK (3): Трекинг в норме>
#7 Timestamp=6.00 → Result=<StateItem OK (3): Трекинг в норме>
#8 Timestamp=7.00 → Result=<StateItem OK (3): Трекинг в норме>
#9 Timestamp=8.00 → Result=<StateItem OK (3): Трекинг в норме>
#10 Timestamp=9.00 → Result=<StateItem OK (3): Трекинг в норме>
#11 Timestamp=10.00 → Result=<StateItem OK (3): Трекинг в норме>
#12 Timestamp=11.00 → Result=<StateItem OK (3): Трекинг в норме>
#13 Timestamp=12.00 → Result=<StateItem OK (3): Трекинг в норме>
#14 Timestamp=13.00 → Result=<StateItem OK (3): Трекинг в норме>
#15 Timestamp=14.00 → Result=<StateItem OK (3):

In [40]:
map = process.slam.map
viz = MapVisualizer(map)

In [41]:
map

<Map | points=960, keyframes=11>

In [42]:
# map.get_points()

In [43]:
map.get_keyframes()

[<KeyFrame id=6, frame_id=22, mps=237, time=0.000> pose t=[0. 0. 0.],
 <KeyFrame id=7, frame_id=23, mps=249, time=1.000> pose t=[-0.026  0.034 -0.999],
 <KeyFrame id=8, frame_id=26, mps=153, time=3.000> pose t=[-0.042  0.071 -3.024],
 <KeyFrame id=9, frame_id=29, mps=151, time=5.000> pose t=[-0.122  0.212 -5.052],
 <KeyFrame id=10, frame_id=32, mps=208, time=7.000> pose t=[-0.164  0.326 -7.094],
 <KeyFrame id=11, frame_id=35, mps=249, time=9.000> pose t=[-0.215  0.493 -9.193],
 <KeyFrame id=12, frame_id=38, mps=266, time=11.000> pose t=[ -0.289   0.647 -11.246],
 <KeyFrame id=13, frame_id=41, mps=216, time=13.000> pose t=[ -0.394   1.03  -13.204],
 <KeyFrame id=14, frame_id=44, mps=210, time=15.000> pose t=[ -0.475   0.958 -15.218],
 <KeyFrame id=15, frame_id=47, mps=184, time=17.000> pose t=[ -0.471   1.103 -17.248],
 <KeyFrame id=16, frame_id=50, mps=145, time=19.000> pose t=[ -2.632   1.67  -20.962]]

In [44]:
viz.show_scene()

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Неверный дескриптор. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Требуемая операция преобразования не поддерживается. 


In [45]:
kfs = process.slam.map.get_keyframes()
kfs

[<KeyFrame id=6, frame_id=22, mps=237, time=0.000> pose t=[0. 0. 0.],
 <KeyFrame id=7, frame_id=23, mps=249, time=1.000> pose t=[-0.026  0.034 -0.999],
 <KeyFrame id=8, frame_id=26, mps=153, time=3.000> pose t=[-0.042  0.071 -3.024],
 <KeyFrame id=9, frame_id=29, mps=151, time=5.000> pose t=[-0.122  0.212 -5.052],
 <KeyFrame id=10, frame_id=32, mps=208, time=7.000> pose t=[-0.164  0.326 -7.094],
 <KeyFrame id=11, frame_id=35, mps=249, time=9.000> pose t=[-0.215  0.493 -9.193],
 <KeyFrame id=12, frame_id=38, mps=266, time=11.000> pose t=[ -0.289   0.647 -11.246],
 <KeyFrame id=13, frame_id=41, mps=216, time=13.000> pose t=[ -0.394   1.03  -13.204],
 <KeyFrame id=14, frame_id=44, mps=210, time=15.000> pose t=[ -0.475   0.958 -15.218],
 <KeyFrame id=15, frame_id=47, mps=184, time=17.000> pose t=[ -0.471   1.103 -17.248],
 <KeyFrame id=16, frame_id=50, mps=145, time=19.000> pose t=[ -2.632   1.67  -20.962]]

In [46]:
K = process.slam.tracking.camera.get_intrinsics()

viz.show_matches(
    kf_ref=kfs[4],
    kf_cur=kfs[5],
    K=K,
    window_name="Initialization Debug",
    font_scale=0.5,       # размер текста для номеров
    wait_key=True          # нажми 'q' для закрытия окна
)


array([[[147, 147, 147],
        [147, 147, 147],
        [150, 150, 150],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[146, 146, 146],
        [149, 149, 149],
        [155, 155, 155],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[151, 151, 151],
        [155, 155, 155],
        [158, 158, 158],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[ 50,  50,  50],
        [ 55,  55,  55],
        [ 54,  54,  54],
        ...,
        [ 23,  23,  23],
        [ 23,  23,  23],
        [ 22,  22,  22]],

       [[ 55,  55,  55],
        [ 60,  60,  60],
        [ 59,  59,  59],
        ...,
        [ 23,  23,  23],
        [ 24,  24,  24],
        [ 26,  26,  26]],

       [[ 64,  64,  64],
        [ 69,  69,  69],
        [ 76,  76,  76],
        ...,
        [ 24,  24,  24],
        [ 24,  24,  24],
        [ 21,  21,  21]]